In [2]:
import pandas as pd

## Get MCBS

In [3]:
mcbs = pd.read_csv("../data/csv/nicoalco.csv")

In [4]:
len(mcbs)

12297

In [5]:
df = pd.read_csv("../data/csv/demo.csv") # demographics file

/n/helmod/apps/centos7/Core/Anaconda3/2020.11-jupyterood-fasrc01/x/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (24,25,26,27,28,29,30,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
len(df)

14068

## Get the MCBS-MBSF cross-walk

In [7]:
xwalk = pd.read_csv("../data/mcbs-xwalk/mcbs_xwalk_file_res000017155_req010441_2018.dat", names=["xwalk"])

In [8]:
# get MCBS ids from xwalk
xwalk["mcbs"]=xwalk["xwalk"].str[:8]
# get MBSF ids
xwalk["mbsf"]=xwalk["xwalk"].str[8:]
xwalk["mcbs"]=xwalk["mcbs"].astype(int)

In [9]:
xwalk=xwalk.drop(columns=['xwalk'])

## Merge MCBS and MBSF cross-walk

In [10]:
mcbs_x = mcbs.merge(xwalk, left_on='BASEID', right_on='mcbs', how='inner')

In [11]:
df_x = df.merge(xwalk, left_on='BASEID', right_on='mcbs', how='inner')

In [12]:
len(mcbs_x)

12297

In [13]:
len(df_x)

14068

## Get MBSF (enrollment)

All MBSF columns:

cols = ["zip","year","qid","dodflag","bene_dod","sex","race","age","hmo_mo","hmoind","statecode","latitude","longitude",
        "dual","death","dead","entry_age","entry_year","entry_age_break","followup_year","followup_year_plus_one","pm25_ensemble",
        "pm25_no_interp","pm25_nn","ozone","ozone_no_interp","zcta","poverty","popdensity","medianhousevalue","pct_blk",
        "medhouseholdincome","pct_owner_occ","hispanic","education","population","zcta_no_interp","poverty_no_interp",
        "popdensity_no_interp","medianhousevalue_no_interp","pct_blk_no_interp","medhouseholdincome_no_interp",
        "pct_owner_occ_no_interp","hispanic_no_interp","education_no_interp","population_no_interp","smoke_rate","mean_bmi",
        "smoke_rate_no_interp","mean_bmi_no_interp","amb_visit_pct","a1c_exm_pct","amb_visit_pct_no_interp",
        "a1c_exm_pct_no_interp","tmmx","rmax","pr","cluster_cat","fips_no_interp","fips","summer_tmmx",
        "summer_rmax","winter_tmmx","winter_rmax"]

In [15]:
cols = ["zip","year","qid","dodflag","bene_dod","sex","race","age","hmo_mo","hmoind","statecode",
        "dual","death","dead","entry_age","entry_year","entry_age_break","followup_year"]

In [ ]:
mbsf2015 = pd.read_csv(
    "../data/denom_by_year_csv/confounder_exposure_merged_nodups_health_2015.csv", usecols=cols)

Merge onto MCBS nicoalco and demo:

In [ ]:
mcbs_x_mbsf = mcbs_x.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2015"))
# An inner merge leads to: len(mcbs_x_mbsf) =  10042

In [ ]:
df_x_mbsf = df_x.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2015"))
# An inner merge leads to: len(df_x_mbsf) =  11585

Get MBSF 2016 and merge onto the merged MCBS x MBSF 2015

In [32]:
mbsf = pd.read_csv(
    "../data/denom_by_year_csv/confounder_exposure_merged_nodups_health_2016.csv", usecols=cols)

In [33]:
mcbs_x_mbsf = mcbs_x_mbsf.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2016"))

In [34]:
df_x_mbsf = df_x_mbsf.merge(
    mbsf, left_on='mbsf', right_on='qid', how='left', suffixes=(None, "_2016"))

In [35]:
len(mcbs_x_mbsf)

12297

In [36]:
len(df_x_mbsf)

14068

## Cleanup and save

In [42]:
mcbs_x_mbsf = mcbs_x_mbsf.drop(columns=['mcbs', 'mbsf','qid_2016'])
df_x_mbsf = df_x_mbsf.drop(columns=['mcbs', 'mbsf','qid_2016'])

Apply "age" >= 65 in 2015 cut

In [62]:
# mcbs_x_mbsf["age"]=pd.to_numeric(mcbs_x_mbsf["age"])
mcbs_x_mbsf = mcbs_x_mbsf[mcbs_x_mbsf["age"] >= 65]

Select the same cohort in demo:

In [69]:
len(mcbs_x_mbsf)

10042

In [63]:
df_x_mbsf = df_x_mbsf[
    df_x_mbsf["BASEID"].isin(mcbs_x_mbsf["BASEID"])]

In [68]:
mcbs_x_mbsf.to_csv("../data/mcbs_nicoalco_x_mbsf_15-16.csv", index=False)
df_x_mbsf.to_csv("../data/mcbs_demo_x_mbsf_15-16.csv", index=False)